# Math

In [2]:
def add(evm):
    a, b = evm.stack.pop().value, cpu.stack.pop().value
    evm.stack.push(a+b)
    evm.pc += 1
    evm.gas_dec(3)

In [3]:
def mul(evm):
    a, b = evm.stack.pop().value, evm.stack.pop().value
    evm.stack.push(a*b)
    evm.pc += 1
    evm.gas_dec(5)